In [2]:
import os
import shutil

In [3]:
base_image_path = 'brisc2025/segmentation_task/train/images'
nameImage = 'Sorted_Images'
folders_set = set()

bip = os.listdir(base_image_path)

for b in bip:
    remove_jpg =  b.replace(".jpg", "")
    split_b = remove_jpg.split('_')
    folder = '_'.join(split_b[3:6])
    folders_set.add(folder)
    
    if not os.path.isdir(nameImage):
        os.mkdir(nameImage)
        print(f"Folder '{nameImage}' created.") 

for folder in folders_set:
        sub = os.path.join(nameImage, folder)
        os.makedirs(sub, exist_ok=True)
        print(f"Folder '{sub}' created successfully.") 

In [4]:
base_mask_path = 'brisc2025/segmentation_task/train/masks'
nameMask = 'Sorted_Masks'
folders_set = set()

bip = os.listdir(base_mask_path)

for b in bip:
    remove_jpg =  b.replace(".jpg", "")
    split_b = remove_jpg.split('_')
    folder = '_'.join(split_b[3:6])
    folders_set.add(folder)
    print(folders_set)
    if not os.path.isdir(nameMask):
        os.mkdir(nameMask)
        print(f"Folder '{nameMask}' created.") 

for folder in folders_set:
        sub = os.path.join(nameMask, folder)
        os.makedirs(sub, exist_ok=True)
        print(f"Folder '{sub}' created successfully.") 

In [5]:
source_d = 'brisc2025/segmentation_task/train/masks' #change for basedir
dest_d = 'Sorted_Masks/pi_sa_t1' #change accordingly for subdir

for images in os.listdir(source_d):
    s_path = os.path.join(source_d, images)
    d_path = os.path.join(dest_d, images)

    if os.path.isfile(s_path):
        if images.__contains__('pi_sa_t1'): #chnage accordingly for subdir
            shutil.move(s_path, d_path)
            print(f"Moved '{images}' to '{d_path}'")

In [ ]:
sort_images = 'Sorted_Images'
sort_masks = 'Sorted_Masks'


for dirpath, dirname, files in os.walk(sort_images):
    for f1 in files:
        sort_image_path = os.path.join(dirpath, f1) 
        
for dirpath, dirname, files in os.walk(sort_masks):
    for f2 in files:
        sort_mask_path = os.path.join(dirpath, f2)
        split_smp = sort_mask_path.split('_')
        code_numberM = split_smp[5]
    
    for f in files:
        if f.__contains__(code_numberM):
                print(sort_mask_path)
                print(code_numberM)


In [7]:
numkeys = []

for dirpath, dirname, files in os.walk(sort_images):
    for f in files:
        jpg = f.replace(".jpg", "")
        split = jpg.split('_')
        numkey = split[2:6]
        joined_numkey = '_'.join(numkey)
        numkeys.append(joined_numkey) 

In [ ]:
#d_(x) <- dierectory 
#dn_(x) <- directory name
#f_(x) <- files in direcotry 

batch_paths = []

for (d_m, dn_ms, f_m), (d_i, dn_is, f_i) in zip(os.walk(sort_masks), os.walk(sort_images)):
        for f1, f2 in zip(f_i, f_m):
               full_path_i = os.path.join(d_i, f1)
               full_path_m = os.path.join(d_m, f2)
               
               for n in numkeys:
                       if full_path_i.__contains__(n) and full_path_m.__contains__(n):
                               batch_paths.append((full_path_i, full_path_m))

for batch in batch_paths:
    print(batch)

In [9]:
import torch
from torchvision import transforms 
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:

#IMG and Mask Transformations (resize 224, 224), tensor output
transform_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

transform_mask = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

#storage 
Tensor_Batch = []

for img_path, mask_path in batch_paths:
    
    image = Image.open(img_path)
    mask = Image.open(mask_path)

    #grayscale 
    if image.mode != 'L' or mask.mode != 'L':
        imgG = image.convert('L')
        maskG = mask.convert('L')

        tensorI = transform_img(image)
        tensorM = transform_mask(mask)
        Tensor_Batch.append((tensorI, tensorM))


In [ ]:
#Custom Dataset

class ImageMask(Dataset):

    def __init__(self, batches, transform):
        self.data = batches 
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
       
       image = self.data[idx]

       if self.transform:
           image = self.transform(image)

       return image

In [33]:
custom_ds = ImageMask(Tensor_Batch, transform=True)

batch_size = 16
dl = DataLoader(Tensor_Batch, batch_size= batch_size, shuffle=True )

for batch in dl:
    print(batch)

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0.0078],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0667, 0.0471],
          [0.0000, 0.0000, 0.0000,  ..., 0.1059, 0.1020, 0.0745],
          [0.0000, 0.0000, 0.0000,  ..., 0.0471, 0.0471, 0.0392]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0.0078],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0745, 0.0667, 0.0471],
          [0.0000, 0.0000, 0.0000,  ..., 0.1059, 0.1020, 0.0745],
          [0.0000, 0.0000, 0.0000,  ..., 0.0471, 0.0471, 0.0392]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 